在讲到 tool calling 这一个概念以前，需要提前了解是什么是 `Function calling`。这是一个在 OpenAI 的[官方文档](https://platform.openai.com/docs/guides/function-calling/example-use-cases?api-mode=responses)中提出的概念。简单来说，`Function Calling` 是指大语言模型在理解用户输入后，自动选择要调用的函数，并填充调用参数，然后等待外部系统调用这个函数，再把结果传回模型继续对话或回答。

具体而言，有以下几个步骤：
1. 对函数进行定义。就如同 3-1 讲的那样，需要定义一个 tool，包含着一定的 schema，向 LLM 声明可以使用的工具，例如有个获取到天气数据的 tool：
```json
functions = [
    {
        "name": "get_weather",
        "description": "Get the weather for a given city.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The name of the city"
                }
            },
            "required": ["location"]
        }
    }
]
```

2. 用户进行提问：
```json
messages = [
    {"role": "user", "content": "What’s the weather like in Paris today?"}
]
```

3. 由 LLM 自动决定是否需要使用额外的 tool，以及使用哪一个 tool
```json
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_weather",
    "arguments": "{ \"location\": \"Paris\" }"
  }
}
```
此时 LLM 需要 tool (`get_weather`) 去查询天气情况，才能回答我们天气到底怎么样。

4. 后端（开发者）根据 LLM 需要的函数名、参数去调用函数，完成 function calling：
```python
# LLM 指明了需要这个函数
def get_weather(location):
    """Fetching the weather data from some websites..."""
    return f"The weather in {location} is XXXXXX."

# LLM 根据用户的查询目标，向函数中传入 "Paris" 这个参数
tool_result = get_weather("Paris")
```
当后端获取到真实的数据之后，我们需要将其作为信息传入 LLM 中：
```python
messages.append({
    "role": "function",
    "name": "get_weather",
    "content": "The weather in Paris is sunny."
})
```

5. LLM 接收到了 role 为 "function" 的消息（包含着天气信息），此时根据 messages 内容 LLM 就可以回答你天气具体怎么样了。




In [2]:
# 老样子，导入我们的配置文件和 API Key
from dotenv import load_dotenv, find_dotenv
from langchain_core.messages import HumanMessage

_ = load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,  # 此时需要我们的 LLM 更严谨
    max_tokens=2048,
    timeout=None,
    max_retries=2,
)

LangChain 中的 tool calling流程：
![Tool Calling Workflow](https://python.langchain.com/assets/images/tool_calling_components-bef9d2bcb9d3706c2fe58b57bf8ccb60.png)
1. 工具创建：使用 @tool 注解，并详细在 description 中解释工具的用法
2. 工具绑定：把工具绑定在支持 tool calling 的 LLM 上
3. 工具调用：如果 LLM 决定时机已到，会调用某一项工具并确保输入输出符合工具的定义
4. 工具执行：LLM 提供参数，由 tool 函数去返回 LLM 所需的结果

接下来我们创建一个具体的工具。我们都知道 LLM（尤其是 GPT-3.5-turbo）在回答 "9.11 和 9.9 哪一个更大？" 这类问题时犯错。为了不它出错，我们亲手设计一个 Comparison 函数，让 LLM 见到这类问题的时候能够想起来用一用我们的 tool，从而得到一个正确的结果。

In [30]:
from langchain_core.messages import HumanMessage
# 我们先问问看 GPT-3.5 这个问题的结果

query = "9.11 和 9.9 哪一个更大？"
message = [
    HumanMessage(content=query)
]
response = llm.invoke(message)
print(response.content)

9.11 更大。


不出所望，LLM 在使用语言模型比较数值问题的时候会犯错。我们接下来创建一个 tool 来让 LLM 使用。

In [31]:
from langchain_core.tools import tool

@tool
def compare(a: float, b: float) -> float:
    """Compare two float number, a anb b
    If a >= b, then return a;
    Else if a < b, the return b.
    """
    return a if a >= b else b

# 我们先尝试一下自己手动调用
compare.invoke({"a": 9.11, "b": 9.9})

9.9

In [32]:
# 我们使用 bind_tool() 方法来绑定工具
llm_with_tools = llm.bind_tools([
    compare
])

response = llm_with_tools.invoke(message)
print(response.content)

Wait，为什么 LLM 用了工具之后没有返回结果？结果是空的？

In [33]:
response.tool_calls

[{'name': 'compare',
  'args': {'a': 9.11, 'b': 9.9},
  'id': 'call_Grj2J8cR5SMxAXDPr7ShtyMt',
  'type': 'tool_call'}]

可以看见 LLM 确实理解了我们给它的工具，并且它也打算调用。但是 LLM 没办法使用这个函数（因为这个函数在本地，我们只给它了这个函数的用法），它在等待我们后端调用这个函数，并把结果返回给它。

In [34]:
from langchain_core.messages import ToolMessage

first_response = response

# 把 assistant 回复（含 tool_calls）也加入 message 中，只有这样我们 LLM 才能知道我们 tool 返回的结果是回答哪一个问题的
message.append(first_response)

# 拿到工具调用请求
tool_call = first_response.tool_calls[0]
tool_name = tool_call['name']
tool_args = tool_call['args']

# 然后我们自己执行工具 compare()
tool_result = compare.invoke(tool_args)

# 我们把工具返回的结果前后文加入到下一次 message 中
message.append(
    ToolMessage(tool_call_id=tool_call['id'], content=str(tool_result))
)

# 最后一步：再次调用 LLM，获取最终答案
final_response = llm_with_tools.invoke(message)
print(final_response.content)

9.11 和 9.9 中，9.9 更大。


OK，此时我们完成了 tool calling 的一个简单用法。但是有个小问题，假设我们 tool 工具类逐渐增多，作为开发人员的我们没法记住那么多 tool 的用法。在这里我们其实是根据 `tool_name = tool_call['name']` 的结果（其实就是 `compare`），去手动调用 `tool_result = compare.invoke(tool_args)` 函数的。

🤔 我们思考一下能否使用类似于 Java 的 Spring Boot 中的反射机制（虽然并非 Spring 特有），让后端自动的根据返回的 `tool_name` 去找到对应的工具呢？

我们可以使用LangChain 或 OpenAI Function Calling 的“自动 Tool 路由器”机制，来自动调用函数。
> 旧版本是用 `ToolExecutor`，根据 `tool_name` 自动匹配你注册的工具并执行。
>
> 新版本我们使用 LangChain `AgentExecutor`。它是一个更高级的封装，自动完成以下流程：
> 1. 让模型决定是否调用工具
> 2. 自动执行工具（无需你自己 .invoke(...)）
> 3. 把工具执行结果传回模型
> 4. 让模型继续生成最终回复

In [44]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_openai_functions_agent, AgentExecutor

# 1. 定义你的工具
tools = [compare]

# 2. 【必需】新版本需要显式传入一个 ChatPromptTemplate，不能自动生成默认 prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个擅长调用工具的助手"),
    # MessagesPlaceholder(variable_name="chat_history"), <-- 注释掉这个即没有上下文记忆，不过对于当前问题没有影响。解除注释会报错，因为没有 chat_history
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# 3. 创建 Agent（使用 OpenAI Function Calling 模式）
agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)

# 4. 创建 executor（自动完成调用/执行/回应）
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

# 5. 一句话调用，自动完成所有流程
response = agent_executor.invoke(
    {"input": "请比较 9.11 和 9.9 哪个更大？"}
)

print(response["output"])

9.9 比 9.11 更大。


Agent 的知识我们会在后面再仔细研究，这里只当做体验一下更高级的 LangChain 流程。